In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/backpack/sample_submission.csv
/kaggle/input/backpack/train.csv
/kaggle/input/backpack/test.csv
/kaggle/input/backpack/training_extra.csv


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Load datasets
train = pd.read_csv("/kaggle/input/backpack/train.csv")
test = pd.read_csv("/kaggle/input/backpack/test.csv")
train_extra = pd.read_csv("/kaggle/input/backpack/training_extra.csv")

In [11]:
import sklearn
print(sklearn.__version__)

1.2.2


In [3]:
train = pd.concat([train, train_extra], ignore_index=True)

In [4]:
# Handle missing values
categorical_cols = ["Brand", "Material", "Size", "Laptop Compartment", "Waterproof", "Style", "Color"]
numerical_cols = ["Compartments", "Weight Capacity (kg)"]

In [5]:
# Fill missing categorical values with mode
def fill_mode(df, columns):
    for col in columns:
        df[col] = df[col].fillna(df[col].mode()[0])

fill_mode(train, categorical_cols)
fill_mode(test, categorical_cols)

In [6]:
# Fill missing numerical values with median
def fill_median(df, columns):
    for col in columns:
        df[col] = df[col].fillna(df[col].median())

fill_median(train, numerical_cols)
fill_median(test, numerical_cols)

In [7]:
# Encode categorical variables
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])
    label_encoders[col] = le

# Define features and target
features = categorical_cols + numerical_cols
X = train[features]
y = train["Price"]
X_test = test[features]

# Standardize numerical features
scaler = StandardScaler()
X.loc[:, numerical_cols] = scaler.fit_transform(X[numerical_cols])
X_test.loc[:, numerical_cols] = scaler.transform(X_test[numerical_cols])

In [8]:
# from sklearn.model_selection import KFold
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error
# import numpy as np
# import joblib
# import pandas as pd

# # Define the best hyperparameters
# best_params = {
#     'n_estimators': 200,  # Optimized number of trees
#     'max_depth': 8,  # Optimized max depth
#     'min_samples_split': 3,  # Optimized min samples to split
#     'min_samples_leaf': 9,  # Optimized min samples per leaf
#     'max_features': 'log2',  # Optimized feature selection strategy
#     'bootstrap': False  # Optimized bootstrapping option
# }

# # Define the number of folds
# n_folds = 5
# kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

# # Initialize Random Forest Regressor with optimized parameters
# rf_model = RandomForestRegressor(
#     n_estimators=best_params['n_estimators'],
#     max_depth=best_params['max_depth'],
#     min_samples_split=best_params['min_samples_split'],
#     min_samples_leaf=best_params['min_samples_leaf'],
#     max_features=best_params['max_features'],
#     bootstrap=best_params['bootstrap'],
#     random_state=42,
#     n_jobs=4
# )

# # Store Out-of-Fold (OOF) predictions
# oof_preds = np.zeros(len(X))  # Initialize array for OOF predictions

# # DataFrame to store out-of-fold predictions
# oof_df = pd.DataFrame(columns=['ID', 'Actual', 'OOF_Pred_RF', 'Fold'])

# # Cross-validation loop
# for fold, (train_idx, val_idx) in enumerate(kf.split(X, y), start=1):
#     print(f"🔄 Training Fold {fold}/{n_folds}...")

#     # Split into training and validation sets
#     X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
#     y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

#     # Train the model on this fold
#     rf_model.fit(X_tr, y_tr)

#     # Predict on validation set
#     y_val_pred = rf_model.predict(X_val)
#     oof_preds[val_idx] = y_val_pred  # Store OOF predictions

#     # Compute RMSE for this fold
#     fold_rmse = mean_squared_error(y_val, y_val_pred, squared=False)
#     print(f"✅ Fold {fold} RMSE: {fold_rmse:.4f}")

#     # Store fold results in DataFrame
#     fold_df = pd.DataFrame({
#         'ID': X.index[val_idx],  # Assuming index represents unique IDs
#         'Actual': y_val.values,
#         'OOF_Pred_RF': y_val_pred,
#         'Fold': fold
#     })
#     oof_df = pd.concat([oof_df, fold_df], ignore_index=True)

# # Compute overall Out-of-Fold (OOF) RMSE
# oof_rmse = mean_squared_error(y, oof_preds, squared=False)
# print(f"\n🏆 Overall OOF RMSE: {oof_rmse:.4f}")

# # Save Out-of-Fold predictions
# oof_df.to_csv('oof_predictions_rf.csv', index=False)
# print("📂 OOF predictions saved to 'oof_predictions_rf.csv'.")

# # Save the final trained Random Forest model
# joblib.dump(rf_model, 'random_forest_model.pkl')
# print("✅ Final trained model saved as 'random_forest_model.pkl'.")

# # Display first few rows of OOF predictions
# oof_df.reset_index(drop=True, inplace=True)
# print(oof_df.head()) 

In [9]:
# Make predictions
y_pred = rf_model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"Validation RMSE: {rmse:.4f}")

# Predict on test dataset
test_predictions = rf_model.predict(X_test)

# Ensure no negative predictions
test_predictions = np.maximum(test_predictions, 0)

# Create submission file
submission = pd.DataFrame({"id": test["id"], "Price": test_predictions})
submission.to_csv("submission_random_forest1.csv", index=False)

print("Improved submission file saved as 'submission_random_forest.csv'")

NameError: name 'rf_model' is not defined

In [ ]:
# import optuna
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import KFold, cross_val_score
# from sklearn.metrics import mean_squared_error
# import numpy as np

# # Define the objective function for Optuna
# def objective(trial):
#     # Define the hyperparameter space to optimize
#     params = {
#         'n_estimators': trial.suggest_int('n_estimators', 100, 500, step=100),  # Number of trees
#         'max_depth': trial.suggest_int('max_depth', 3, 20),  # Maximum depth of trees
#         'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),  # Minimum samples to split
#         'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),  # Minimum samples per leaf
#         'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),  # Feature selection
#         'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),  # Whether to use bootstrapping
#     }

#     # Number of folds for cross-validation
#     n_folds = 3
#     kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

#     # Perform cross-validation
#     rf_model = RandomForestRegressor(random_state=42, n_jobs=-1, **params)
#     scores = cross_val_score(rf_model, X, y, cv=kf, scoring='neg_root_mean_squared_error', n_jobs=-1)

#     # Optuna minimizes the objective function, so we return the negative RMSE
#     return -np.mean(scores)

# # Run Optuna optimization
# study = optuna.create_study(direction='minimize')  # Minimizing RMSE
# study.optimize(objective, n_trials=3, n_jobs=-1)  # Perform 50 trials

# # Best hyperparameters
# best_params = study.best_params
# print(f"\n🏆 Best Parameters Found: {best_params}")
